## 데이터 수집

* archives.go.kr
* http://theme.archives.go.kr/next/closed/page/ReadClosedCommon.do?yr=2019

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import time
import numpy as np

In [2]:
url = "http://theme.archives.go.kr/next/closed/page/ReadClosedCommon.do?yr=2019"

In [3]:
response = requests.get(url)
html = bs(response.text)

In [4]:
table = pd.read_html(url)
len(table)

207

In [5]:
# body > table > tbody > tr:nth-child(2) > td:nth-child(2) > table > tbody > tr:nth-child(4) > td > table
url = "http://theme.archives.go.kr/next/closed/page/orgListClosedCommon.do"
query = "bookmark=&pageParam=page&page=2&searchResultCount=&gubun=80&orgnz_id=108001&searchType=&keyword=&keywordGubun=&viewPage=ClosedList2017"

In [6]:
q_split = query.split("&")
params = {}
for q in q_split:
    q_key = q.split("=")[0]
    q_val = q.split("=")[1]
    params[q_key] = q_val
    
params

{'bookmark': '',
 'pageParam': 'page',
 'page': '2',
 'searchResultCount': '',
 'gubun': '80',
 'orgnz_id': '108001',
 'searchType': '',
 'keyword': '',
 'keywordGubun': '',
 'viewPage': 'ClosedList2017'}

In [7]:
def requests_page(url, params):
    response = requests.post(url, params)
    response.raise_for_status()
    html = bs(response.text)
    return html

In [8]:
def find_last_page_no(html):
    find_last_page = html.select("a")[-1]["href"]
    last_page_no = re.sub("[^0-9]", "", find_last_page)
    last_page_no = int(last_page_no)
    return last_page_no

In [9]:
def parse_table(html):
    table = html.select("table")
    doc = table[1].select("table")[2]
    df_doc = pd.read_html(str(doc))[0]
    cols = df_doc.loc[0].tolist()
    df_doc.columns = cols
    df_doc = df_doc.drop(0)
    return df_doc

In [10]:
def get_table(url, params):
    """
    url, params 를 넘기면 기록물 목록을 반환
    """
    html = requests_page(url, params)
    table = parse_table(html)
    return table

In [11]:
html = requests_page(url, params)
last_page_no = find_last_page_no(html)
last_page_no

34988

In [12]:
params

{'bookmark': '',
 'pageParam': 'page',
 'page': '2',
 'searchResultCount': '',
 'gubun': '80',
 'orgnz_id': '108001',
 'searchType': '',
 'keyword': '',
 'keywordGubun': '',
 'viewPage': 'ClosedList2017'}

In [13]:
last_page_no = 3
page_list = []
for page_no in range(1, last_page_no+1):
    """
    0~1초 휴지시간을 갖고 페이지마다 돌며 수집
    """
    random_time = np.random.uniform(0, 1)
    time.sleep(random_time)
    
    params["page"] = page_no
    df_page = get_table(url, params)
    page_list.append(df_page)
    df = pd.concat(page_list)
    df.to_csv("archives_go_kr.csv", index=False)

In [14]:
print(df.shape)
pd.read_csv("archives_go_kr.csv").head()

(30, 7)


,번호,기록물건명,생산부서,기록물철명,생산년도,관리번호,재분류결과
0,1,"사건송치(경계침범,명예훼손)",경찰청 강원도지방경찰청 고성경찰서 수사과,수사사건종결철 2 (1980),1980,CA0319141,부분공개
1,2,사건송치(사기),경찰청 강원도지방경찰청 고성경찰서 수사과,수사사건종결철 2 (1980),1980,CA0319141,부분공개
2,3,사건송치(폭행),경찰청 강원도지방경찰청 고성경찰서 수사과,수사사건종결철 2 (1980),1980,CA0319141,부분공개
3,4,"사건송치(명예훼손,폭행)",경찰청 강원도지방경찰청 고성경찰서 수사과,수사사건종결철 2 (1980),1980,CA0319141,부분공개
4,5,"사건송치(사기,배임,강제집행면탈)",경찰청 강원도지방경찰청 고성경찰서 수사과,수사사건종결철 2 (1980),1980,CA0319141,부분공개
